In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font", family=font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

import seaborn as sns
from sklearn.svm import SVR, SVC

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

In [3]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LinearRegression

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

import statsmodels.api as sm
from sklearn import preprocessing

In [57]:
df1 = pd.read_excel("평점크롤링1.xlsx", index_col = False)
df2 = pd.read_excel("평점크롤링2.xlsx", index_col = False)
df3 = pd.read_excel("평점크롤링3.xlsx", index_col = False)
df4 = pd.read_excel("평점크롤링4.xlsx", index_col = False)

In [58]:
df5 = pd.read_csv("중간저장9.csv", index_col = False, encoding = "utf-8")

In [59]:
df = pd.concat([df1, df2, df3, df4])
df.reset_index(inplace = True)
df.drop("index",axis=1, inplace=True)

In [60]:
df5["평점"] = df["별점"]
df5["리뷰수"] = df["참여자수"]

In [61]:
df5.to_csv("중간저장13.csv", index = False, encoding = "utf-8-sig")

In [4]:
df6 = pd.read_csv("중간저장13.csv", index_col = False, encoding = "utf-8")

In [5]:
# 개봉일 날짜 형식으로 변경
df6["개봉일"]=  pd.to_datetime(df6["개봉일"], format="%Y-%m-%d")

In [6]:
# 개봉일 없는 데이터 삭제
df6 = df6[df6["개봉일"] != datetime.strptime("1900-01-01", "%Y-%m-%d")]

In [7]:
# 배급사 배우 감독 모두 0인 데이터 삭제
df6 = df6[(df6["배우합"] != 0 ) | (df6["감독값"] != 0 ) |(df6["배급사값"] != 0 )]

In [8]:
df6

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,...,캐나다,프랑스,한국,12세이상관람가,15세이상관람가,등급없음,전체관람가,청소년관람불가,평점,리뷰수
1,미씨카: 더 네크로맨서,2017-11-02,300000,300000,60,60,1,1,미국,(주)제이에이와이이엔터테인먼트,...,0,0,0,0,1,0,0,0,0.0,0
2,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,기타,주식회사풍경소리,...,0,0,0,0,1,0,0,0,0.0,0
3,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사루믹스미디어,...,0,0,0,0,1,0,0,0,0.0,0
5,연쇄살인마의 이중생활,2020-06-18,16000,16000,7,7,5,7,스페인,주식회사마운틴픽쳐스,...,0,0,0,0,1,0,0,0,0.0,0
6,몬스터 아일랜드,2017-10-27,3000,3000,1,1,1,1,미국,주식회사레드아이스,...,0,0,0,0,1,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21314,화끈한 새엄마,2019-11-28,6000,6000,1,1,1,1,일본,(주)영진크리에이티브,...,0,0,0,0,0,0,0,1,0.0,0
21315,블랙 가스펠,2013-11-14,367328500,367328500,53217,53217,143,2933,한국,(주)넥스트엔터테인먼트월드(NEW),...,0,0,1,0,0,0,1,0,0.0,0
21318,패밀리 S다이어리,2014-04-24,2992500,2992500,601,601,5,13,기타,주식회사풍경소리,...,0,0,0,0,0,0,0,1,0.0,0
21322,16세의 사운드트랙,2020-07-02,3543400,3543400,413,413,19,150,영국,(주)마노엔터테인먼트,...,0,0,0,1,0,0,0,0,0.0,0


In [37]:
df.columns

Index(['영화명', '개봉일', '매출액', '누적매출액', '관객수', '누적관객수', '스크린수', '상영횟수', '대표국적',
       '배급사', '등급', '장르', '감독', '감독값', '배급사값', '배우합', '배우평균', 'SF', '가족', '공연',
       '공포', '느와르', '다큐멘터리', '드라마', '멜로로맨스', '모험', '무협', '뮤지컬', '미스터리', '범죄',
       '사극', '서부', '서사', '스릴러', '애니메이션', '액션', '어드벤처', '에로', '장르없음', '전쟁',
       '코미디', '판타지', '기타', '독일', '미국', '스페인', '영국', '이탈리아', '일본', '중국', '캐나다',
       '프랑스', '한국', '12세이상관람가', '15세이상관람가', '등급없음', '전체관람가', '청소년관람불가', '평점',
       '리뷰수'],
      dtype='object')

In [9]:
del df6['영화명']
del df6['대표국적']
del df6['등급']
del df6['장르']
del df6["배우평균"]
del df6["개봉일"]
del df6["매출액"]
del df6["관객수"]
del df6["배급사"]
del df6["감독"]

In [12]:
from statsmodels.formula.api import ols

In [10]:
X = df6.drop(['누적관객수', "누적매출액"], axis = 1)
# , 'SF', '가족', '공연',
#        '공포', '느와르', '다큐멘터리', '드라마', '멜로로맨스', '모험', '무협', '뮤지컬', '미스터리', '범죄',
#        '사극', '서부', '서사', '스릴러', '애니메이션', '액션', '어드벤처', '에로', '장르없음', '전쟁',
#        '코미디', '판타지', '기타', '독일', '미국', '스페인', '영국', '이탈리아', '일본', '중국', '캐나다',
#        '프랑스', '한국', '12세이상관람가', '15세이상관람가', '등급없음', '전체관람가', '청소년관람불가'], axis = 1)
y = df6['누적관객수']

In [11]:
X = sm.add_constant(X, has_constant = "add")

In [13]:
multi_model = sm.OLS(y, X)
fitted_multi_model = multi_model.fit()
fitted_multi_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  누적관객수   R-squared:                       0.703
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     669.0
Date:                Thu, 19 Aug 2021   Prob (F-statistic):               0.00
Time:                        00:48:54   Log-Likelihood:            -1.8025e+05
No. Observations:               12495   AIC:                         3.606e+05
Df Residuals:                   12450   BIC:                         3.609e+05
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.696e+04   1.86e+04      3.069      0.002    2.06e+04    9.33e+04
스크린수       -1590.0358     48.046    -33.094      0.000   -1684.213   -1495.859
상영횟수          33.0527      0.657     50.313      0.000      31.765      34.340
감독값            0.1625      0.008     19.352      0.000       0.146       0.179
배급사값           0.2617      0.013     20.905      0.000       0.237       0.286
배우합            0.0305      0.003     10.134      0.000       0.025       0.036
SF         -9638.8037   3.55e+04     -0.272      0.786   -7.92e+04    5.99e+04
가족         -8.816e+04   5.24e+04     -1.684      0.092   -1.91e+05    1.45e+04
공연         -1.353e+05   4.47e+04     -3.027      0.002   -2.23e+05   -4.77e+04
공포         -4.089e+04   2.89e+04     -1.413      0.158   -9.76e+04    1.58e+04
느와르         4.618e+04    4.3e+05      0.108      0.914   -7.96e+05    8.88e+05
다큐멘터리      -1.231e+05   3.25e+04     -3.788      0.000   -1.87e+05   -5.94e+04
드라마        -5.748e+04   2.42e+04     -2.376      0.018   -1.05e+05   -1.01e+04
멜로로맨스      -2.604e+04   2.57e+04     -1.011      0.312   -7.65e+04    2.44e+04
모험         -3.857e+04   9.42e+04     -0.409      0.682   -2.23e+05    1.46e+05
무협         -5522.0652   2.49e+05     -0.022      0.982   -4.93e+05    4.82e+05
뮤지컬        -8.566e+04   7.41e+04     -1.157      0.247   -2.31e+05    5.95e+04
미스터리       -9.908e+04   4.17e+04     -2.377      0.017   -1.81e+05   -1.74e+04
범죄           1.41e+04   3.36e+04      0.420      0.675   -5.18e+04       8e+04
사극          6.757e+05   6.48e+04     10.424      0.000    5.49e+05    8.03e+05
서부          3.724e+04   9.85e+04      0.378      0.706   -1.56e+05     2.3e+05
서사          1.415e-10   1.05e-09      0.135      0.893   -1.92e-09     2.2e-09
스릴러        -2.511e+04   3.09e+04     -0.812      0.417   -8.57e+04    3.55e+04
애니메이션        9.26e+04      3e+04      3.089      0.002    3.38e+04    1.51e+05
액션          2.282e+04   2.58e+04      0.886      0.376   -2.77e+04    7.33e+04
어드벤처       -5.006e+04   5.18e+04     -0.966      0.334   -1.52e+05    5.15e+04
에로         -5054.8318   2.78e+04     -0.182      0.856   -5.95e+04    4.94e+04
장르없음       -4.133e+04   3.62e+04     -1.141      0.254   -1.12e+05    2.97e+04
전쟁          3.665e+04   4.88e+04      0.751      0.453    -5.9e+04    1.32e+05
코미디        -4.841e+04   2.72e+04     -1.779      0.075   -1.02e+05    4942.189
판타지         1.104e+04   4.31e+04      0.256      0.798   -7.35e+04    9.56e+04
기타         -8561.8408   1.54e+04     -0.555      0.579   -3.88e+04    2.17e+04
독일          2.753e+04   2.86e+04      0.962      0.336   -2.86e+04    8.36e+04
미국          2.332e+04   1.08e+04      2.151      0.031    2069.437    4.46e+04
스페인         6979.1827   3.45e+04      0.202      0.840   -6.07e+04    7.47e+04
영국          7680.0786   2.03e+04      0.378      0.706   -3.22e+04    4.75e+04
이탈

In [14]:
X.corr(method = "pearson")

,const,스크린수,상영횟수,감독값,배급사값,배우합,SF,가족,공연,공포,...,캐나다,프랑스,한국,12세이상관람가,15세이상관람가,등급없음,전체관람가,청소년관람불가,평점,리뷰수
const,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
스크린수,NaN,1.000000,0.850873,0.262235,0.561233,0.412898,0.033294,3.280993e-03,-0.030818,-0.001787,...,-0.021478,-0.028192,0.065142,0.132098,0.095683,-0.078192,0.110424,-0.213674,0.197879,0.397630
상영횟수,NaN,0.850873,1.000000,0.249031,0.479130,0.364009,0.033209,-5.457528e-03,-0.024403,-0.006782,...,-0.022105,-0.034128,0.096359,0.111310,0.083880,-0.054943,0.027537,-0.145935,0.166480,0.440511
감독값,NaN,0.262235,0.249031,1.000000,0.278958,0.312447,0.041555,-8.738578e-04,-0.012682,-0.016582,...,-0.014158,-0.021936,0.056076,0.078567,0.039572,-0.030571,0.003956,-0.079482,0.141996,0.407135
배급사값,NaN,0.561233,0.479130,0.278958,1.000000,0.425164,0.038806,1.181122e-02,-0.021583,0.005558,...,-0.027641,-0.041894,0.080471,0.161992,0.079929,-0.088618,0.118582,-0.221643,0.224413,0.399055
배우합,NaN,0.412898,0.364009,0.312447,0.425164,1.000000,0.004624,1.940757e-02,-0.032166,-0.039706,...,-0.022783,-0.051667,0.254480,0.113777,0.122149,-0.082389,0.011450,-0.159117,0.192022,0.365405
SF,NaN,0.033294,0.033209,0.041555,0.038806,0.004624,1.000000,-1.187353e-02,-0.015272,-0.032643,...,0.056280,-0.022677,-0.068682,0.072766,0.070404,-0.029801,-0.020489,-0.087205,0.028010,0.063555
가족,NaN,0.003281,-0.005458,-0.000874,0.011811,0.019408,-0.011874,1.000000e+00,-0.008810,-0.018830,...,-0.008682,0.019027,0.020454,0.013316,-0.026002,-0.014192,0.133765,-0.065612,0.038569,-0.005326
공연,NaN,-0.030818,-0.024403,-0.012682,-0.021583,-0.032166,-0.015272,-8.809803e-03,1.000000,-0.024220,...,-0.003839,-0.013302,-0.034714,0.040980,-0.041249,-0.024075,0.173751,-0.092951,-0.039358,-0.022262
공포,NaN,-0.001787,-0.006782,-0.016582,0.005558,-0.039706,-0.032643,-1.883011e-02,-0.024220,1.000000,...,0.040313,-0.035734,-0.074440,-0.067839,0.091289,-0.048018,-0.079879,0.039492,-0.048212,-0.021049


In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [17]:
movie_model1 = LinearRegression()
movie_model1.fit(X_train, y_train)

print("훈련 점수 : ", movie_model1.score(X_train, y_train))
print("테스트 점수 : ", movie_model1.score(X_test, y_test))

훈련 점수 :  0.7069352104932054
테스트 점수 :  0.6917104367938489


In [18]:
import lightgbm as lgb

In [19]:
model1 = lgb.LGBMRegressor(random_state=1, n_estimators=1000)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 14)

params = {
    'learning_rate': [0.1, 0.01, 0.001, 0.003],
    'min_child_samples': [10, 20]
}
# kfold = KFold(n_splits = 5)

grid_model = GridSearchCV(estimator = model1,
                          param_grid = params,
                          scoring = 'r2',
                          cv = 5,
                          refit = True,
                             verbose = 100)

grid_model.fit(X_train, y_train)

print("최적의 파라미터 : ", grid_model.best_params_)
print("최고 정확도 : ", grid_model.best_score_)
print(grid_model.score(X_test, y_test))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] learning_rate=0.1, min_child_samples=10 .........................
[CV]  learning_rate=0.1, min_child_samples=10, score=0.695, total=   2.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s
[CV] learning_rate=0.1, min_child_samples=10 .........................
[CV]  learning_rate=0.1, min_child_samples=10, score=0.675, total=   2.4s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.5s remaining:    0.0s
[CV] learning_rate=0.1, min_child_samples=10 .........................
[CV]  learning_rate=0.1, min_child_samples=10, score=0.720, total=   1.6s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    6.2s remaining:    0.0s
[CV] learning_rate=0.1, min_child_samples=10 .........................
[CV]  learning_rate=0.1, min_child_samples=10, score=0.579, total=   1.5s
[Parallel(n_jobs=1)]: Done   4 out of  

[CV]  learning_rate=0.003, min_child_samples=20, score=0.744, total=   1.4s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:  1.0min remaining:    0.0s
[CV] learning_rate=0.003, min_child_samples=20 .......................
[CV]  learning_rate=0.003, min_child_samples=20, score=0.718, total=   1.4s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:  1.1min remaining:    0.0s
[CV] learning_rate=0.003, min_child_samples=20 .......................
[CV]  learning_rate=0.003, min_child_samples=20, score=0.631, total=   1.4s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:  1.1min remaining:    0.0s
[CV] learning_rate=0.003, min_child_samples=20 .......................
[CV]  learning_rate=0.003, min_child_samples=20, score=0.694, total=   1.5s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  1.1min finished
최적의 파라미터 :  {'learning_rate': 0.01, 'min_child_samples': 20}
최고 정확도 :  0.7095014656898452
0.7

In [21]:
grid_model.score(X_test, y_test)

0.7381032015704437

In [24]:
import xgboost as xgb
model= xgb.XGBRegressor(colsample_bytree=0.4603, gamma=1, 
                             learning_rate=0.0025, max_depth=10, 
                             min_child_weight=0.5, n_estimators=5000,
                             subsample=0.35,
                             random_state =70, nthread = -1,
                             scale_pos_weight=1 )

model.fit(X_train, y_train)

print("훈련",model.score(X_train, y_train))
print("테스트",model.score(X_test, y_test))

훈련 0.9784967664436293
테스트 0.7301671312111873


In [25]:
model.score(X_test, y_test)

0.7301671312111873